Lago SDK Example - one VM one Network
----------------------------------------------

In [1]:
import logging
import tempfile
from textwrap import dedent
from lago import sdk

Create a LagoInitFile, normally this file should be saved to the disk. Here we will use a temporary file instead. Our environment includes one CentOS 7.3 VM with one network.

In [6]:
with tempfile.NamedTemporaryFile(delete=False) as init_file:
    init_file.write(dedent("""
    domains:
      vm-01:
        memory: 1024
        nics:
          - net: net-01
        disks:
          - template_name: el7.3-base
            type: template
            name: root
            dev: sda
            format: qcow2
    nets:
      net-01:
        type: nat
        dhcp:
          start: 100
          end: 254
        """))

Now we will initialize the environment by using the init file. Our ``workdir`` will be created automatically if it does not exists. If this is the first time you are running Lago, it might take a while as it will download the CentOS 7.3 template. You can monitor its progress by watching the log file we configured in ``/tmp/lago.log``. 

In [7]:
env = sdk.init(config=init_file.name,
              workdir='/tmp/lago_sdk_simple_example',
              loglevel=logging.DEBUG,
              log_fname='/tmp/lago.log')

When the method returns, the environment can be started:

In [8]:
env.start()

Check which VMs are available and get some meta data:

In [10]:
vms = env.get_vms()
print vms

{'vm-01': <lago.vm.DefaultVM object at 0x7f7375db6050>}


In [13]:
vm = vms['vm-01']


'192.168.202.2'

In [14]:
vm.distro()


u'el7'

In [19]:
vm.ip()

'192.168.202.2'

Executing commands in the VM can be done with ``ssh`` method:

In [20]:
res = vm.ssh(['hostname', '-f'])

In [21]:
res

CommandStatus(code=0, out='vm-01.lago.local\n', err='')

Lets stop the environment, here we will use the ``destroy`` method, however you may also use ``stop`` and ``start`` if you would like to turn the environment off. 

In [22]:
env.destroy()